In [1]:
from cdc.src.FastText import *

set_keras_backend("tensorflow")
data_home = '/Users/zebinkang/Github/scalable_text_analysis/' # os.getcwd()
wemb_file=data_home+'/test/data/crawl-300d-2M.vec'

train(data_home+'test/data/data.txt', 
     data_home+'test/data/label.txt',
     data_home+'test/data/wiki-news-300d-1M.vec')

/anaconda2/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


start
--- Loading data ---
Read 431 rows of data
Read 431 rows of label
--- Label encoding ---
--- Text processing ---


/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Found 999995 word vectors.
--- Build model ---


/Users/zebinkang/Github/scalable_text_analysis/cdc/src/FastText.py:194: UserWarning: Update your `GRU` call to the Keras 2 API: `GRU(128, return_sequences=True, implementation=1)`
  m = Bidirectional(GRU(128, return_sequences=True, consume_less='mem'))(embedded_sequences)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 300, 300)          3741300   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 300, 256)          329472    
_________________________________________________________________
attention_1 (Attention)      (None, 256)               556       
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 1542      
Total params: 4,072,870
Trainable params: 331,570
Non-trainable params: 3,741,300
_________________________________________________________________
None
--- Training ---
Train on 302 samples, validate on 129 samples
Epoch 1/100
302/302 [==============================] - 5s 

302/302 [==============================] - 4s 12ms/step - loss: 0.2681 - acc: 0.9139 - precision: 0.9468 - recall: 0.8940 - fmeasure: 0.9195 - val_loss: 0.3145 - val_acc: 0.8992 - val_precision: 0.9095 - val_recall: 0.8605 - val_fmeasure: 0.8843
Epoch 29/100
302/302 [==============================] - 4s 12ms/step - loss: 0.2089 - acc: 0.9437 - precision: 0.9590 - recall: 0.9238 - fmeasure: 0.9411 - val_loss: 0.3726 - val_acc: 0.8915 - val_precision: 0.9319 - val_recall: 0.8605 - val_fmeasure: 0.8947
Epoch 30/100
302/302 [==============================] - 4s 12ms/step - loss: 0.1931 - acc: 0.9536 - precision: 0.9655 - recall: 0.9338 - fmeasure: 0.9493 - val_loss: 0.4106 - val_acc: 0.8915 - val_precision: 0.9167 - val_recall: 0.8682 - val_fmeasure: 0.8917
Epoch 31/100
302/302 [==============================] - 4s 12ms/step - loss: 0.1593 - acc: 0.9669 - precision: 0.9697 - recall: 0.9536 - fmeasure: 0.9615 - val_loss: 0.2486 - val_acc: 0.9225 - val_precision: 0.9415 - val_recall: 0.8915 

NameError: name 'cPickle' is not defined